In [10]:
import os
import pandas as pd
import math
import copy
import numpy as np
from tqdm import tqdm

In [11]:

def fixations_saccades_detection(samples:pd.DataFrame, min_pursuit_dur:float=10., max_pso_dur:float=0.0, min_fix_dur:float=0.05, 
                                 sac_max_vel:float=1000., fix_max_amp:float=1.5, sac_time_thresh:float=0.002,
                                 drop_fix_from_blink:bool=True, sfreq:float=1000,
                                 screen_size:float=38., screen_resolution:int=1920, screen_distance:float=60,
                                 out_fname:str='events', out_folder:str='Remodnav_detection/'):

    """
    Detects fixations and saccades from eye-tracking data for both left and right eyes.

    Parameters
    ----------
    samples : pd.DataFrame
        DataFrame containing eye-tracking data with columns 'LX' (left gaze x-coordinate), 'LY' (left gaze y-coordinate), 'LPupil' (left pupil size),
        'RX' (right gaze x-coordinate), 'RY' (right gaze y-coordinate), and 'RPupil' (right pupil size).
    min_pursuit_dur : float, optional
        Minimum pursuit duration in seconds for Remodnav detection (default is 10.0).
    max_pso_dur : float, optional
        Maximum post-saccadic oscillation duration in seconds for Remodnav detection (default is 0.0 -No PSO events detection-).
    min_fix_dur : float, optional
        Minimum fixation duration in seconds for Remodnav detection (default is 0.05).
    sac_max_vel : float, optional
        Maximum saccade velocity in deg/s (default is 1000.0).
    fix_max_amp : float, optional
        Maximum fixation amplitude in deg (default is 1.5).
    sac_time_thresh : float, optional
        Time threshold in seconds to consider a saccade as neighboring a fixation (default is 0.002).
    drop_fix_from_blink : bool, optional
        Whether to drop fixations that do not have a previous saccade within the time threshold (default is True).
    sfreq : float, optional
        Sampling frequency of the eye-tracking data in Hz (default is 1000).
    screen_size : float, optional
        Size of the screen in cm (default is 38.0).
    screen_resolution : int, optional
        Horizontal resolution of the screen in pixels (default is 1920).
    screen_distance : float, optional
        Distance from the screen to the participant's eyes in cm (default is 60).
    out_fname : str, optional
        Name of the output file (without extension) to save Remodnav detection results (default is 'events').
    out_folder : str, optional
        Name of the folder to save the output files (default is 'Remodnav_detection/').

    Returns
    -------
    fixations : dict
        Dictionary containing DataFrames of detected fixations for both left and right eyes with additional columns for mean x, y positions, and pupil size.
    saccades : dict
        Dictionary containing DataFrames of detected saccades for both left and right eyes.
    times : np.ndarray
        Array of time points corresponding to the samples.

    Raises
    ------
    ValueError
        If Remodnav detection fails.
    """
    
    times = np.arange(stop=len(samples) / sfreq, step=1/sfreq)

    # Dictionaries to store fixations and saccades dataframes from each eye
    fixations = {}
    saccades = {}

    for gazex_data, gazey_data, pupil_data, eye in zip((samples['LX'], samples['RX']), 
                                                       (samples['LY'], samples['RY']), 
                                                       (samples['LPupil'], samples['RPupil']), 
                                                       ('left', 'right')):

        # If not pre run data, run
        print(f'\nRunning eye movements detection for {eye} eye...')

        # Define data to save to excel file needed to run the saccades detection program Remodnav
        eye_data = {'x': gazex_data, 'y': gazey_data}
        df = pd.DataFrame(eye_data)

        # Remodnav parameters
        eye_samples_fname = f'eye_samples_{eye}.csv'  # eye data file to use as input for Remodnav 
        px2deg = math.degrees(math.atan2(.5 * screen_size, screen_distance)) / (.5 * screen_resolution)  # Pixel to degree conversion
        results_fname = out_fname + f'_{eye}.tsv'  # Output results filename 
        image_fname = out_fname + f'_{eye}.png'  # Output image filename

        # Save csv file
        df.to_csv(eye_samples_fname, sep='\t', header=False, index=False)

        # Run Remodnav not considering pursuit class and min fixations 50 ms
        command = f'remodnav {eye_samples_fname} {results_fname} {px2deg} {sfreq} --min-pursuit-duration {min_pursuit_dur} ' \
                    f'--max-pso-duration {max_pso_dur} --min-fixation-duration {min_fix_dur} --max-vel {sac_max_vel}'
        failed = os.system(command)

        # Raise error if events detection with Remodnav failed
        if failed:
            raise ValueError('Remodnav detection failed')

        # Read results file with detections
        sac_fix = pd.read_csv(results_fname, sep='\t')

        # Move eye data, detections file and image to subject results directory
        os.makedirs(out_folder, exist_ok=True)
        # Move et data file
        os.replace(eye_samples_fname, out_folder + eye_samples_fname)
        # Move results file
        os.replace(results_fname, out_folder + results_fname)
        # Move results image
        os.replace(image_fname, out_folder + image_fname)

        # Get saccades and fixations
        saccades_eye_all = copy.copy(sac_fix.loc[(sac_fix['label'] == 'SACC') | (sac_fix['label'] == 'ISAC')])
        fixations_eye_all = copy.copy(sac_fix.loc[sac_fix['label'] == 'FIXA'])

        # Drop saccades and fixations based on conditions
        print(f'Dropping saccades with average vel > {sac_max_vel} deg/s, and fixations with amplitude > {fix_max_amp} deg')

        fixations_eye = copy.copy(fixations_eye_all[(fixations_eye_all['amp'] <= fix_max_amp)])
        saccades_eye = copy.copy(saccades_eye_all[saccades_eye_all['peak_vel'] <= sac_max_vel])

        print(f'Kept {len(fixations_eye)} out of {len(fixations_eye_all)} fixations')
        print(f'Kept {len(saccades_eye)} out of {len(saccades_eye_all)} saccades')

        # Variables to save fixations features
        mean_x = []
        mean_y = []
        pupil_size = []
        prev_sac = []
        next_sac = []

        # Identify neighbour saccades to each fixation (considering sac_time_thresh)
        print('Finding previous and next saccades')

        for fix_idx, fixation in tqdm(fixations_eye.iterrows(), total=len(fixations_eye)):

            fix_time = fixation['onset']
            fix_dur = fixation['duration']

            # Previous and next saccades
            try:
                sac0 = saccades_eye.loc[(saccades_eye['onset'] + saccades_eye['duration'] > fix_time - sac_time_thresh) & (
                            saccades_eye['onset'] + saccades_eye['duration'] < fix_time + sac_time_thresh)].index.values[-1]
            except:
                sac0 = None
            prev_sac.append(sac0)

            try:
                sac1 = saccades_eye.loc[(saccades_eye['onset'] > fix_time + fix_dur - sac_time_thresh) & (
                            saccades_eye['onset'] < fix_time + fix_dur + sac_time_thresh)].index.values[0]
            except:
                sac1 = None
            next_sac.append(sac1)


        # Add columns
        fixations_eye['prev_sac'] = prev_sac
        fixations_eye['next_sac'] = next_sac

        # Drop when no previous saccade detected in sac_time_thresh
        if drop_fix_from_blink:
            fixations_eye.dropna(subset=['prev_sac'], inplace=True)
            print(f'\nKept {len(fixations_eye)} fixations with previous saccade')


        # Fixations features
        print('Computing average pupil size, and x and y position')

        for fix_idx, fixation in tqdm(fixations_eye.iterrows(), total=len(fixations_eye)):

            fix_time = fixation['onset']
            fix_dur = fixation['duration']

            # Average pupil size, x and y position
            fix_time_idx = np.where(np.logical_and(times > fix_time, times < fix_time + fix_dur))[0]

            pupil_data_fix = pupil_data[fix_time_idx]
            gazex_data_fix = gazex_data[fix_time_idx]
            gazey_data_fix = gazey_data[fix_time_idx]

            pupil_size.append(np.nanmean(pupil_data_fix))
            mean_x.append(np.nanmean(gazex_data_fix))
            mean_y.append(np.nanmean(gazey_data_fix))

        fixations_eye['mean_x'] = mean_x
        fixations_eye['mean_y'] = mean_y
        fixations_eye['pupil'] = pupil_size
        fixations_eye = fixations_eye.astype({'mean_x': float, 'mean_y': float, 'pupil': float, 'prev_sac': 'Int64', 'next_sac': 'Int64'})

        fixations[eye] = fixations_eye
        saccades[eye] = saccades_eye

    return fixations, saccades, times

In [13]:
# ----- Example ----- #

# Get path to samples from parsed edf
cwd = os.getcwd()
parent_directory = os.path.abspath(os.path.join(cwd, os.pardir))
path_to_samples = parent_directory + '/example_dataset_derivatives/sub-ab01/ses-second/samples.hdf5'

# Load samples.hdf5 file
samples = pd.read_hdf(path_or_buf=path_to_samples)

# Run eye movements detection and save results
fixations, saccades, time = fixations_saccades_detection(samples=samples)

# Show output for left eye fixations
fixations['left']


Running eye movements detection for left eye...
Dropping saccades with average vel > 1000.0 deg/s, and fixations with amplitude > 1.5 deg
Kept 1806 out of 2030 fixations
Kept 1840 out of 1840 saccades
Finding previous and next saccades


100%|██████████| 1806/1806 [00:01<00:00, 1544.25it/s]



Kept 1695 fixations with previous saccade
Computing average pupil size, and x and y position


100%|██████████| 1695/1695 [00:02<00:00, 663.06it/s]



Running eye movements detection for right eye...
Dropping saccades with average vel > 1000.0 deg/s, and fixations with amplitude > 1.5 deg
Kept 1848 out of 2001 fixations
Kept 1795 out of 1795 saccades
Finding previous and next saccades


100%|██████████| 1848/1848 [00:01<00:00, 1561.65it/s]



Kept 1696 fixations with previous saccade
Computing average pupil size, and x and y position


100%|██████████| 1696/1696 [00:02<00:00, 703.26it/s]


,onset,duration,label,start_x,start_y,end_x,end_y,amp,peak_vel,med_vel,avg_vel,prev_sac,next_sac,mean_x,mean_y,pupil
3,0.679,0.152,FIXA,865.75953,396.03525,856.08797,404.79620,0.239,14.084,6.042,6.347,2,4,860.772848,403.846358,636.152318
5,0.870,0.141,FIXA,931.17497,477.55971,955.24719,506.97957,0.696,20.989,7.562,8.434,4,6,942.266429,497.467857,625.821429
7,1.030,1.687,FIXA,971.06356,532.38448,972.64790,570.41570,0.697,21.298,5.301,5.871,6,8,973.263938,545.931020,531.162515
9,2.728,0.276,FIXA,961.54631,557.59646,960.21782,559.76028,0.046,22.469,7.042,7.995,8,10,967.050000,553.152174,506.760870
11,3.015,0.841,FIXA,979.20575,554.12455,966.49089,543.94830,0.298,30.483,4.970,5.492,10,12,971.323571,549.935119,518.244048
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3861,826.445,0.260,FIXA,486.16086,778.50279,481.36342,776.42795,0.096,14.359,4.830,5.083,3860,3862,493.305405,770.683784,536.590734
3863,826.727,0.470,FIXA,504.34445,854.07077,495.51031,834.43525,0.394,68.369,4.695,6.558,3862,3864,496.251386,837.158635,568.933902
3865,827.246,0.421,FIXA,444.69253,870.65440,459.55228,865.79752,0.286,20.736,4.875,5.683,3864,3866,453.079286,855.956667,589.261905
3867,827.684,0.579,FIXA,512.29973,871.23963,483.14414,863.22711,0.553,49.608,6.337,7.861,3866,3868,490.378201,854.664360,595.942907
